In [ ]:
pip install pandas google-cloud-storage

In [ ]:
import requests
import pandas as pd
import time
import os
from google.cloud import bigquery

#  Authentification GCP
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/project-final-laka-93110-e98c1369c8cc.json"

#Extraction globale
PAGE_SIZE = 1000
NUM_PAGES = 20
CSV_PATH = "/content/openfood_referentiel.csv"

# BigQuery
PROJECT_ID = "project-final-laka-93110"
DATASET_ID = "Laka10"
TABLE_ID = "openfoodfacts"
BQ_TABLE = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

#  Vérification de connexion à l'API
def check_api_connection():
    try:
        response = requests.get("https://world.openfoodfacts.org/", timeout=5)
        if response.status_code == 200:
            print(" Connexion à l'API OpenFoodFacts établie.")
            return True
    except Exception as e:
        print(f" Échec de connexion à l'API : {e}")
    return False

# Fonction : récupération sécurisée des produits
def fetch_all_products(page, page_size=1000):
    url = "https://world.openfoodfacts.org/cgi/search.pl"
    params = {
        "action": "process",
        "page_size": page_size,
        "page": page,
        "json": True,
    }
    try:
        res = requests.get(url, params=params, timeout=10)
        res.raise_for_status()
        return res.json().get("products", [])
    except requests.exceptions.RequestException as e:
        print(f"Erreur HTTP page {page} : {e}")
    except ValueError as e:
        print(f"Erreur JSON page {page} : {e}")
    return []

# Extraction d'infos produit
def extract_product_info(product):
    return {
        "product_name": product.get("product_name", ""),
        "brands": product.get("brands", ""),
        "stores": product.get("stores", ""),
        "nutriscore_grade": product.get("nutriscore_grade", ""),
        "nutrition_score_fr": product.get("nutrition_score_fr", ""),
        "energy_kcal": product.get("nutriments", {}).get("energy-kcal_100g"),
        "fat_100g": product.get("nutriments", {}).get("fat_100g"),
        "saturated_fat_100g": product.get("nutriments", {}).get("saturated-fat_100g"),
        "sugars_100g": product.get("nutriments", {}).get("sugars_100g"),
        "salt_100g": product.get("nutriments", {}).get("salt_100g"),
        "fiber_100g": product.get("nutriments", {}).get("fiber_100g"),
        "proteins_100g": product.get("nutriments", {}).get("proteins_100g"),
        "labels": product.get("labels", ""),
        "origins": product.get("origins", ""),
        "categories": product.get("categories", ""),
        "url": product.get("url", ""),
        "code": product.get("code", ""),
    }

def save_to_csv(df, path):
    df.to_csv(path, index=False)
    print(f"Fichier CSV sauvegardé : {path}")

def load_to_bigquery(csv_path, table_id):
    client = bigquery.Client()
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        autodetect=True,
        write_disposition="WRITE_TRUNCATE"
    )
    with open(csv_path, "rb") as source_file:
        job = client.load_table_from_file(source_file, table_id, job_config=job_config)
    job.result()
    print(f"Données chargées dans BigQuery : {table_id}")

def get_data_from_bigquery(table_id):
    client = bigquery.Client()
    query = f"SELECT * FROM `{table_id}`"
    df = client.query(query).to_dataframe()
    print(f" {len(df)} lignes récupérées depuis BigQuery")
    return df

def transform_data(df):
    if df.empty:
        print(" DataFrame vide : aucune transformation possible.")
        return df

    print(" Transformation des données...")

    df['energy_kcal'] = pd.to_numeric(df['energy_kcal'], errors='coerce')
    df['fat_100g'] = pd.to_numeric(df['fat_100g'], errors='coerce')
    df['sugars_100g'] = pd.to_numeric(df['sugars_100g'], errors='coerce')
    df['nutriscore_grade'] = df['nutriscore_grade'].str.upper()
    df['product_name'] = df['product_name'].fillna("Inconnu")
    df['has_label_bio'] = df['labels'].str.contains("bio", case=False, na=False)

    df["nutrient_density"] = df["energy_kcal"] / (
        df["proteins_100g"].astype(float).fillna(0) +
        df["fat_100g"].astype(float).fillna(0) +
        df["sugars_100g"].astype(float).fillna(0)
    ).replace(0, 1)

    print(" Données transformées.")
    return df

#  Main pipeline
def main():
    if not check_api_connection():
        print(" Arrêt du pipeline car l’API OpenFoodFacts est injoignable.")
        return

    all_products = []
    for page in range(1, NUM_PAGES + 1):
        print(f" Téléchargement page {page}/{NUM_PAGES}")
        products = fetch_all_products(page, PAGE_SIZE)
        if not products:
            print(f"Page {page} vide ou invalide. Passage à la suivante.")
            continue
        all_products.extend([extract_product_info(p) for p in products])
        time.sleep(1)

    df = pd.DataFrame(all_products)
    print(f"{len(df)} produits extraits.")

    save_to_csv(df, CSV_PATH)
    load_to_bigquery(CSV_PATH, BQ_TABLE)

    bq_df = get_data_from_bigquery(BQ_TABLE)
    transformed_df = transform_data(bq_df)
    transformed_df.to_csv("/content/openfood_transformed.csv", index=False)
    print("Données transformées sauvegardées.")

if __name__ == "__main__":
    main()


 Connexion à l'API OpenFoodFacts établie.
 Téléchargement page 1/20
 Téléchargement page 2/20
 Téléchargement page 3/20
 Téléchargement page 4/20
 Téléchargement page 5/20
 Téléchargement page 6/20
 Téléchargement page 7/20
 Téléchargement page 8/20
 Téléchargement page 9/20
 Téléchargement page 10/20
 Téléchargement page 11/20
🔴 Erreur HTTP page 11 : 429 Client Error: Too Many Requests for url: https://world.openfoodfacts.org/cgi/search.pl?action=process&page_size=1000&page=11&json=True
⚠️ Page 11 vide ou invalide. Passage à la suivante.
 Téléchargement page 12/20
🔴 Erreur HTTP page 12 : 429 Client Error: Too Many Requests for url: https://world.openfoodfacts.org/cgi/search.pl?action=process&page_size=1000&page=12&json=True
⚠️ Page 12 vide ou invalide. Passage à la suivante.
 Téléchargement page 13/20
🔴 Erreur HTTP page 13 : 429 Client Error: Too Many Requests for url: https://world.openfoodfacts.org/cgi/search.pl?action=process&page_size=1000&page=13&json=True
⚠️ Page 13 vide ou inva

In [ ]:

# === Étape 1 : Chargement du fichier CSV ===

original_csv_path = "/content/openfood_referentiel.csv"
cleaned_csv_path = "/content/openfood_referentiel_cleaned.csv"

df = pd.read_csv(original_csv_path)

# Nettoyage des noms de colonnes
def clean_column_name(col):
    col = col.strip()
    col = re.sub(r"[^\w]", "_", col)
    col = re.sub(r"__+", "_", col)
    return col.strip("_")

df.columns = [clean_column_name(col) for col in df.columns]

# === Étape 2 : Transformation des données ===

print("Début des transformations...")

# Convertir les colonnes numériques
numeric_cols = [
    "energy_kcal", "fat_100g", "saturated_fat_100g",
    "sugars_100g", "salt_100g", "fiber_100g", "proteins_100g",
    "nutrition_score_fr"
]
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Valeurs manquantes et nettoyage de base
df["product_name"] = df["product_name"].fillna("Inconnu")
df["nutriscore_grade"] = df["nutriscore_grade"].fillna("").str.upper()
for col in ["labels", "brands", "categories", "origins", "stores"]:
    df[col] = df[col].fillna("").str.lower().str.strip()

# Dictionnaires de traduction
translations = {
    "organic": "bio",
    "gluten-free": "sans gluten",
    "vegetarian": "végétarien",
    "vegan": "végétalien",
    "non-gmo": "sans OGM",
    "halal": "halal",
    "kosher": "kasher",
    "beverages": "boissons",
    "dairies": "produits laitiers",
    "sodas": "sodas",
    "snacks": "snacks",
    "cereals": "céréales",
    "meats": "viandes",
    "ready-meals": "plats préparés",
    "breakfasts": "petits-déjeuners",
    "cheeses": "fromages",
    "desserts": "desserts",
    "france": "France",
    "germany": "Allemagne",
    "italy": "Italie",
    "spain": "Espagne",
    "carrefour": "Carrefour",
    "leclerc": "Leclerc",
    "lidl": "Lidl",
    "auchan": "Auchan",
    "monoprix": "Monoprix"
}

def translate_text(text):
    for eng, fr in translations.items():
        text = re.sub(rf"\b{eng}\b", fr, text, flags=re.IGNORECASE)
    return text

# Dictionnaire de translittération arabe → français
arabic_to_french = {
    "سلطان": "Sultan",
    "الراية": "Al-Raya",
    "كارفور": "Carrefour",
    "أوشان": "Auchan",
    "ليدل": "Lidl"
}

def transliterate_text(text):
    for ar, fr in arabic_to_french.items():
        text = re.sub(rf"\b{ar}\b", fr, text)
    return text

# Appliquer traduction + translittération
for col in ["labels", "categories", "stores", "brands", "origins"]:
    df[col] = df[col].apply(translate_text)
    df[col] = df[col].apply(transliterate_text)
    df[col] = df[col].str.title().str.strip()

# Ajout de colonnes dérivées
df["has_label_bio"] = df["labels"].str.contains("bio", case=False, na=False)

df["nutrient_density"] = df["energy_kcal"] / (
    df["proteins_100g"].fillna(0) +
    df["fat_100g"].fillna(0) +
    df["sugars_100g"].fillna(0)
).replace(0, 1)

def classify_nutriscore(score):
    score = str(score).strip().upper()
    return {
        "A": "Excellent",
        "B": "Bon",
        "C": "Moyen",
        "D": "Médiocre",
        "E": "Mauvais"
    }.get(score, "Inconnu")

df["qualite_nutritionnelle"] = df["nutriscore_grade"].apply(classify_nutriscore)

print("Transformations terminées.")

# === Étape 3 : Sauvegarde du fichier transformé ===

df.to_csv(cleaned_csv_path, index=False)
print(f"Fichier nettoyé sauvegardé : {cleaned_csv_path}")

# === Étape 4 : Chargement dans BigQuery ===

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/project-final-laka-93110-e98c1369c8cc.json"

PROJECT_ID = "project-final-laka-93110"
DATASET_ID = "Laka10"
TABLE_ID = "openfoodfacts"
BQ_TABLE = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

client = bigquery.Client(project=PROJECT_ID)

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
    write_disposition="WRITE_TRUNCATE"
)

with open(cleaned_csv_path, "rb") as source_file:
    load_job = client.load_table_from_file(source_file, BQ_TABLE, job_config=job_config)

load_job.result()

table = client.get_table(BQ_TABLE)
print(f" {table.num_rows} lignes importées dans {BQ_TABLE}.")


🔄 Début des transformations...
✅ Transformations terminées.
✅ Fichier nettoyé sauvegardé : /content/openfood_referentiel_cleaned.csv
 1000 lignes importées dans project-final-laka-93110.Laka10.openfoodfacts.


In [ ]:
# === Étape 1 : Chargement du fichier CSV ===

original_csv_path = "/content/openfood_referentiel.csv"
cleaned_csv_path = "/content/openfood_referentiel_cleaned.csv"

df = pd.read_csv(original_csv_path)

# Nettoyage des noms de colonnes
def clean_column_name(col):
    col = col.strip()
    col = re.sub(r"[^\w]", "_", col)
    col = re.sub(r"__+", "_", col)
    return col.strip("_")

df.columns = [clean_column_name(col) for col in df.columns]

# === Étape 2 : Transformation des données ===

print("Début des transformations...")

# Convertir les colonnes numériques
numeric_cols = [
    "energy_kcal", "fat_100g", "saturated_fat_100g",
    "sugars_100g", "salt_100g", "fiber_100g", "proteins_100g",
    "nutrition_score_fr"
]
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Nettoyage de base
df["product_name"] = df["product_name"].fillna("Inconnu")
df["nutriscore_grade"] = df["nutriscore_grade"].fillna("").str.upper()
for col in ["labels", "brands", "categories", "origins", "stores"]:
    df[col] = df[col].fillna("").str.lower().str.strip()

# Suppression des lignes inutiles ou trop incomplètes
df = df[df["product_name"].str.lower() != "inconnu"]
min_non_null = len(df) * 0.5
df = df.dropna(thresh=min_non_null, axis=1)
df = df.dropna(subset=["energy_kcal", "sugars_100g", "saturated_fat_100g", "proteins_100g", "fiber_100g"])

# Ajout de la colonne calculée
df["scoring_nutritionnel_personnalise"] = (
    df["energy_kcal"] + df["sugars_100g"] + df["saturated_fat_100g"] -
    (df["proteins_100g"] + df["fiber_100g"])
)

# Traductions
translations = {
    "organic": "bio",
    "gluten-free": "sans gluten",
    "vegetarian": "végétarien",
    "vegan": "végétalien",
    "non-gmo": "sans OGM",
    "halal": "halal",
    "kosher": "kasher",
    "beverages": "boissons",
    "dairies": "produits laitiers",
    "sodas": "sodas",
    "snacks": "snacks",
    "cereals": "céréales",
    "meats": "viandes",
    "ready-meals": "plats préparés",
    "breakfasts": "petits-déjeuners",
    "cheeses": "fromages",
    "desserts": "desserts",
    "france": "France",
    "germany": "Allemagne",
    "italy": "Italie",
    "spain": "Espagne",
    "carrefour": "Carrefour",
    "leclerc": "Leclerc",
    "lidl": "Lidl",
    "auchan": "Auchan",
    "monoprix": "Monoprix"
}

def translate_text(text):
    for eng, fr in translations.items():
        text = re.sub(rf"\b{eng}\b", fr, text, flags=re.IGNORECASE)
    return text

arabic_to_french = {
    "سلطان": "Sultan",
    "الراية": "Al-Raya",
    "كارفور": "Carrefour",
    "أوشان": "Auchan",
    "ليدل": "Lidl"
}

def transliterate_text(text):
    for ar, fr in arabic_to_french.items():
        text = re.sub(rf"\b{ar}\b", fr, text)
    return text

for col in ["labels", "categories", "stores", "brands", "origins"]:
    df[col] = df[col].apply(translate_text)
    df[col] = df[col].apply(transliterate_text)
    df[col] = df[col].str.title().str.strip()

# Ajouter une détection du label "bio"
df["has_label_bio"] = df["labels"].str.contains("bio", case=False, na=False)

# Densité nutritionnelle
df["nutrient_density"] = df["energy_kcal"] / (
    df["proteins_100g"].fillna(0) +
    df["fat_100g"].fillna(0) +
    df["sugars_100g"].fillna(0)
).replace(0, 1)

# Classification du nutriscore
def classify_nutriscore(score):
    score = str(score).strip().upper()
    return {
        "A": "Excellent",
        "B": "Bon",
        "C": "Moyen",
        "D": "Médiocre",
        "E": "Mauvais"
    }.get(score, "Inconnu")

df["qualite_nutritionnelle"] = df["nutriscore_grade"].apply(classify_nutriscore)

# === (Optionnel) Traduire les noms de produits localement (commentaire) ===
# from deep_translator import GoogleTranslator
# df["product_name"] = df["product_name"].apply(lambda x: GoogleTranslator(source='auto', target='fr').translate(x))

print("Transformations terminées.")

# === Étape 3 : Sauvegarde ===

df.to_csv(cleaned_csv_path, index=False)
print(f"Fichier nettoyé sauvegardé : {cleaned_csv_path}")

# === Étape 4 : Chargement dans BigQuery ===

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/project-final-laka-93110-e98c1369c8cc.json"

PROJECT_ID = "project-final-laka-93110"
DATASET_ID = "Laka10"
TABLE_ID = "openfoodfacts"
BQ_TABLE = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

client = bigquery.Client(project=PROJECT_ID)

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
    write_disposition="WRITE_TRUNCATE"
)

with open(cleaned_csv_path, "rb") as source_file:
    load_job = client.load_table_from_file(source_file, BQ_TABLE, job_config=job_config)

load_job.result()
table = client.get_table(BQ_TABLE)
print(f"{table.num_rows} lignes importées dans {BQ_TABLE}.")


🔄 Début des transformations...
✅ Transformations terminées.
✅ Fichier nettoyé sauvegardé : /content/openfood_referentiel_cleaned.csv
✅ 613 lignes importées dans project-final-laka-93110.Laka10.openfoodfacts.
